# LLM Database Integration



Want to create some ChatGPT functions that will allow the model to talk directly with the database. The way I will achieve this is using Chat GPTs tool_calls functionality. This allows a user to pass in a list of tools and get the GPT to decide which tool it would like to call. We then use a class and RESTful APIs to call the requests data. 

The challenge here is getting the LLM to answer complex questions. This can be solved with recursion!

We can contiously call the LLM until it is satisfied it has all the data to answer the question. This script will illustrate how this was built with full deployment in app.py in the root directory. First we set up our Open AI client:

In [40]:
import sys
import requests
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

sys.path.append('C:/projects/python/gpt-database-wrapper')

from src.util.database_API_connection import ApplicationAPI
GPT_API_KEY = os.getenv('OPENAI_API_KEY')

api = ApplicationAPI(1)

Test the ApplicationAPI to check that it is working

In [42]:
api.get_all_system_severity_data()

[{'systems': 'Blade Bearing A', 'severity2': 58},
 {'systems': 'Blade Bearing C', 'severity2': 58},
 {'systems': 'Gearbox', 'severity1': 54, 'severity0': 3, 'severity2': 1},
 {'systems': 'Hydraulic Pitch Station',
  'severity1': 30,
  'severity2': 16,
  'severity0': 12},
 {'systems': 'Yaw', 'severity2': 58},
 {'systems': 'Blade Bearing B', 'severity2': 57},
 {'systems': 'Hydraulic System', 'severity0': 2},
 {'systems': 'Blade Bearing A - Inner', 'severity2': 1},
 {'systems': 'Blade Bearing A - Outer', 'severity2': 1},
 {'systems': 'Bearing spongs', 'severity2': 1},
 {'systems': 'Drive End Brg - Outer cover plate', 'severity0': 1},
 {'systems': 'Drive End Brg - Race', 'severity0': 1},
 {'systems': 'Drive End End Shield - Frame', 'severity2': 1},
 {'systems': 'Non Drive End Brg - Race', 'severity0': 1},
 {'systems': 'Hydraulic Pitch Station / Before Filter Frial.', 'severity0': 1},
 {'systems': 'Hydraulic Pitch Station / Filter', 'severity1': 1},
 {'systems': 'Hydraulic Pitch Station / F

An introduction to send to the LLM to provide context

In [49]:
GPT_Introduction = """The system manages a database of reports and assets for various sites. Each site contains multiple assets, and each asset is linked to specific systems. 
Reports generated for these systems contain critical data points, including severity levels that indicate the importance of the information (0=no warning, 1=early warning, 2=advanced warning).
The API provides access to these reports, assets, and system details, enabling in-depth data analysis. 
"""

Create a function that can handle the tool selection made by the LLM

In [43]:
def handle_function_call(function_name, function_args):
    """
    Handles API function calls based on the function name and arguments.
    Updates the context with the data collected and determines the next steps.
    
    Args:
        function_name (str): The name of the function to call.
        function_args (dict): Arguments needed for the API function.
        context (dict): Context of the conversation to be updated.
    
    Returns:
        tuple: The response from the function and the updated context.
    """
    function_response = None
    break_loop = False

    try:
        if function_name == "get_asset_ids_names":
            asset_data = api.get_asset_ids_names()
            function_response = asset_data        

        elif function_name == "get_single_report_data":
            if 'reportID' in function_args:
                report_data = api.get_single_report_data(function_args['reportID'])
                function_response = report_data
          
            else:
                function_response = "Report ID missing."
                
        elif function_name == "get_all_report_data_from_asset_names":
            if 'assetName' in function_args:
                reports_data = api.get_all_report_data_from_asset_names(function_args['assetName'])
                function_response = reports_data

        elif function_name == "get_all_report_data_from_asset_names_full":
            if 'assetName' in function_args:
                reports_data = api.get_all_report_data_from_asset_names_full(function_args['assetName'])
                function_response = reports_data
            
        elif function_name == "get_all_system_severity_data":
            severity_data = api.get_all_system_severity_data()
            function_response = severity_data
        

        elif function_name == "get_number_of_assets":
            num_assets = api.get_number_of_assets()
            function_response = num_assets
        

        elif function_name == "get_number_of_reports":
            num_reports = api.get_number_of_reports()
            function_response = num_reports
        

        elif function_name == "get_all_asset_severity_data":
            severity_data = api.get_all_asset_severity_data()
            function_response = severity_data

        elif function_name == "no_more_data_required":
            break_loop = True
            function_response = []

        else:
            function_response = "Function not recognized."

    except Exception as e:
        function_response = f"An error occurred: {str(e)}"

    return function_response, break_loop

Create a function that provides the LLM with a list of potential tools it can use

In [44]:
def define_tools():
    """
    Dynamically define the list of tools available based on the context of the conversation.
    Args:
    context (dict): The context dictionary containing keys like 'data_collected' and 'data_needed'.
    
    Returns:
    list: A list of tool definitions that should be available for the next step.
    """
    tools = []

    # Adding other fixed tools to the list
    tools.append({
        "type": "function",
        "function": {
            "name": "get_all_system_severity_data",
            "description": "Fetch high level details on the severities of all the systems in the site. Severity2 is advanced warning, Severity1 is early warning, and Severity0 is no warning.",
            "parameters": {},
        }
    })

    tools.append({
        "type": "function",
        "function": {
            "name": "get_number_of_assets",
            "description": "Get number of assets at the site.",
            "parameters": {},
        }
    })

    tools.append({
        "type": "function",
        "function": {
            "name": "get_number_of_reports",
            "description": "Get number of reports at the site.",
            "parameters": {},
        }
    })

    tools.append({
        "type": "function",
        "function": {
            "name": "get_all_asset_severity_data",
            "description": "Asset information sorted from most severe to least severe.",
            "parameters": {},
        }
    })

    tools.append({
        "type": "function",
        "function": {
            "name": "get_all_report_data_from_asset_names",
            "description": "Fetch all reports for a given asset name.",
            "parameters": {
                "type": "object",
                "properties": {
                    "assetName": {"type": "string", "description": "Asset name to fetch reports for"},
                },
                "required": ["assetName"],
            }
        }
    })


    tools.append({
        "type": "function",
        "function": {
            "name": "get_single_report_data",
            "description": "Fetch details of a single report that relates to a single asset and system. This call provides mode information on what is causing problems.",
            "parameters": {
                "type": "object",
                "properties": {
                    "reportID": {"type": "string", "description": "The ID of the report to fetch"},
                },
                "required": ["reportID"],
            }
        }
    })

    tools.append({
        "type": "function",
        "function": {
            "name": "no_more_data_required",
            "description": "Call this function when no more data from other functions would help answer the query",
            "parameters": {},
        }
    })

    return tools


Recursion script, the while loop will continue until max_depth is reached or tool_calls is "stop". When this has happened we are able to parse the result.

In [45]:
from openai import OpenAI
import json

client = OpenAI(
  api_key=GPT_API_KEY,
)


def run_conversation(user_query, introduction, max_depth, session_messages=None):
    if session_messages is None:
        session_messages = []

    # Start the conversation or add to it
    if not session_messages:
        session_messages.append({"role": "system", "content": introduction})
    session_messages.append({"role": "user", "content": user_query})

    # Define tools based on context
    tools = define_tools()
    
    depth = 0  # Initialize depth counter
    while depth < max_depth:
        # Call the GPT model with current session messages and tools
        response = client.chat.completions.create(
            model="gpt-3.5-turbo-0125",
            messages=session_messages,
            tools=tools,
            tool_choice="auto",
        )

        print(response)

        # Check the finish reason of the response
        finish_reason = response.choices[0].finish_reason
        print(f'Response with finish_reason = {finish_reason}')
        
        if finish_reason == "stop":
            # If finish_reason is 'stop', return the response and end the loop
            return response, session_messages
        elif finish_reason == "tool_calls":
            # Handle tool calls and continue the loop
            tool_calls = response.choices[0].message.tool_calls
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments) if tool_call.function.arguments else {}
                function_response = handle_function_call(function_name, function_args)
                session_messages.append({"role": "system", "name": function_name, "content": json.dumps(function_response)})
            depth += 1  # Increment depth after each cycle
        else:
            # Continue if other reasons but log unexpected behavior
            print(f'Unhandled finish reason: {finish_reason}')
            break

    # This point should not be reached if while loop is correctly configured
    return None, session_messages


Use the tool and send a question that gets the LLM to pull disperate peices of information

In [46]:
# Example of running the function with a dynamic query
tools = define_tools()
user_input = f"""What is the most affected system at the site? What is the state of this system for the asset A2? What is the Chromium levels for these systems in the asset?"""

response, updated_messages = run_conversation(user_input, GPT_Introduction, max_depth = 10)

ChatCompletion(id='chatcmpl-9NLLm0Ecc094WYXV6e9u2RZUN6CyF', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_BFrkDgclMdQ8EcLweq1GH09V', function=Function(arguments='{}', name='get_all_system_severity_data'), type='function')]))], created=1715350642, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=14, prompt_tokens=374, total_tokens=388))
Response with finish_reason = tool_calls
ChatCompletion(id='chatcmpl-9NLLnoHt2Hoc0cblf4YmjtRxB5MyC', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zEUtMMMZwrzDVwhmOAYmhZhw', function=Function(arguments='{}', name='get_all_asset_severity_data'), type='function')]))], created=1715350

Parse the result

In [47]:
message_content = response.choices[0].message.content
print(message_content)

The most affected system at the site is "Blade Bearing A" with a severity level of 2. 

For the asset A2:
- Blade Bearing A: 
    - Severity Level: 2
    - Report Date: May 24, 2023
    - Chromium level: 290

- Blade Bearing A - Inner: 
    - Severity Level: 2
    - Report Date: May 4, 2023
    - Chromium level: 149

- Blade Bearing A - Outer: 
    - Severity Level: 2
    - Report Date: May 11, 2023
    - Chromium level: 167

Please note that the Chromium levels for the other systems in the asset A2 are not available in the reports.


## Results

The results show that the recursive algorithmn is able to produce insights by pulling disperate pieces of information together to formulate a result.